In [ ]:
csvfile = r'C:\Users\roumba\Documents\Software\deep-events\Videos\done\220915_mtstaygold_cos7_ZEIS_1\Fluorescent\labels1f.csv'

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import tensorflow
import imageio
from skimage.segmentation import flood, flood_fill
from scipy import signal
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
import myfunctions
from myfunctions import event_separation , image_crop_save_gauss , get_gaussian



datacsv = pd.read_csv(csvfile)
sigma_trial=(8,8)
size_trial=(2048,2048)


In [2]:
def get_gaussian(mu, sigma, size):
    mu = ((mu[1]+0.5-0.5*size[1])/(size[1]*0.5), (mu[0]+0.5-0.5*size[0])/(size[0]*0.5))
    sigma = (sigma[0]/size[0], sigma[1]/size[1])
    mvn = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
    x,y = tf.cast(tf.linspace(-1,1,size[0]),tf.float32), tf.cast(tf.linspace(-1,1,size[1]), tf.float32)
    # meshgrid as a list of [x,y] coordinates
    coords = tf.reshape(tf.stack(tf.meshgrid(x,y),axis=-1),(-1,2)).numpy()
    gauss = mvn.prob(coords)
    return tf.reshape(gauss, size)

In [4]:
points_of_interest= np.zeros((100, 2048, 2048))

for row_number in tqdm(range(0, len(datacsv))):
    framenumber_in_row= int(datacsv.loc[row_number, 'axis-0'])
    fission_ycoord = int(datacsv.loc[row_number, 'axis-1'])
    fission_xcoord = int(datacsv.loc[row_number, 'axis-2'])
    fission_coords=(fission_ycoord,fission_xcoord)
    gaussian_points=get_gaussian(fission_coords,sigma_trial,size_trial)                                     #gets gaussian points at a single frame
    gaussian_points = gaussian_points.numpy()                                                               #convers tensor into numpy array
    gaussian_points = gaussian_points/np.max(gaussian_points)                                               #divides by the max 
    gaussian_points[gaussian_points < 0.1] = 0                                                              #sets background to zero
    gaussian_points = gaussian_points/np.max(gaussian_points)                                               #divides by max again
    points_of_interest[framenumber_in_row] = points_of_interest[framenumber_in_row] + gaussian_points       #adds the gaussian intensity in the empty file 


   
imageio.mimwrite('points1b.tiff', (points_of_interest*254).astype(np.uint8))

100%|██████████| 498/498 [01:45<00:00,  4.73it/s]


In [5]:
list1=event_separation(datacsv)
l=len(list1)
gauss=r'C:\Users\roumba\Documents\Software\deep-events\points1b.tiff'
gauss_image=Image.open(gauss)
image_crop_save_gauss(l,list1, datacsv, gauss_image)

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.